<a href="https://colab.research.google.com/github/patrickrsoerensen/grpassign4/blob/main/groupassign4_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Group assignment 4
Group members: Frederik Kaminski Klitte, Patrick Riber Sørensen, Emmanouil Tzevelekos, Sadik Hasan Khan Chowdhury

#Installs

In [3]:
!pip install --upgrade pip
!pip install accelerate --q
!pip install pypdf --q
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off
!pip install -qqq gradio==3.50.2
!pip install -qqq "pydantic>=1.9,<2.0"

!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch=

#Importing Libraries

In [4]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

from textwrap import fill
import torch

import requests

In [5]:
!locale  # Use in Jupyter/Colab notebook

LANG=en_US.UTF-8
LANGUAGE=en_US
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


#Set-up

Loading our document directly from github repository

In [6]:
pdf_url = "https://github.com/patrickrsoerensen/grpassign4/raw/main/introdeeplearning.pdf"
response = requests.get(pdf_url)

with open("file.pdf", "wb") as file:
    file.write(response.content)

print("PDF file downloaded successfully.")


PDF file downloaded successfully.


In [7]:
loader = PyPDFLoader("/content/file.pdf")

docs = loader.load()
len(docs)

1151

Splitting the document into smaller chunks for faster processing. Chunk size is 1024 characters and overlaps for 64 characters to better preserve context

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

2845

This code initializes a text embedding model using Hugging Face Transformers. It sets parameters for model usage and embedding processing, then embeds the text content of the first document chunk.

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

1024


This code creates a database (db) using the **Chroma library**, which stores embeddings generated from the provided texts. It specifies a directory for persisting the database. Then, it performs a similarity search within the database for documents similar to the query "Advantages", retrieving the top 2 results.

In [10]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("What is deep learning?", k=2)
print(results[0].page_content)

29 Summary
43anyonewishingtolearnaboutdeeplearningandwehopethatourreaderswillbenefitfrom
this.
1.8Summary
Machine learning studies how computer systems can leverage experience (often data) to
improve performance at specific tasks. It combines ideas from statistics, data mining, and
optimization. Often, it is used as a means of implementing AI solutions. As a class of
machine learning, representational learning focuses on how to automatically find the ap-
propriate way to represent data. Considered as multi-level representation learning through
learning many layers of transformations, deep learning replaces not only the shallow mod-
els at the end of traditional machinelearning pipelines, but also the labor-intensiveprocess
of feature engineering. Much of the recent progress in deep learning has been triggered
by an abundance of data arising from cheap sensors and Internet-scale applications, and
by significant progress in computation, mostly through GPUs. Furthermore, the availabil-


Overall, this code sets up a text generation pipeline using a pre-trained language model, tokenizer, and generation configuration, with specific settings for controlling the generated text's properties like randomness and diversity.

In [29]:
MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


#Creating Question answer chain and trying different prompt approaches

##Template 1 and first queries

In [14]:
template1 = """
<s>[INST] <<SYS>>
You are a Data Scientist tasked with analyzing the following information to provide insights for the question at the end. You need to be concise and accurate in your explanation, but try to not convey it to be too advanced
<</SYS>>

{context}

{question} [/INST]
"""

prompt1 = PromptTemplate(template=template1, input_variables=["context", "question"])


qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt1},
)

In [15]:
result1 = qa_chain1("What is deep learning?")

print(fill(result1["result"].strip(),width=80))

Deep learning is a type of machine learning that involves training artificial
neural networks to learn and improve on their own by analyzing large amounts of
data. These neural networks are composed of multiple layers of interconnected
nodes or "neurons," which process and transform the input data in complex ways.
By stacking these layers together, deep learning algorithms can learn to
recognize patterns and make predictions in images, speech, text, and other forms
of data. The key advantage of deep learning is its ability to automatically
extract features from raw data, without requiring manual feature engineering.
This makes it particularly useful for tasks like image recognition, natural
language processing, and autonomous driving, where the amount of available data
is vast and complex. Deep learning has become increasingly popular in recent
years due to advances in computing power and the availability of large datasets.
The use of graphics processing units (GPUs) has made it possib

In [16]:
result2 = qa_chain1("What advantages does deep learning have over traditional machine learning?")

print(fill(result2["result"].strip(),width=80))

Deep learning offers several advantages over traditional machine learning:  1.
**Automatic feature extraction**: Deep learning algorithms learn how to extract
relevant features from raw data without requiring manual engineering. This saves
time and effort compared to traditional machine learning approaches, which often
require extensive feature engineering. 2. **Improved accuracy**: By learning
multiple layers of representations, deep learning models can capture complex
patterns in the data, leading to improved accuracy compared to traditional
machine learning models. 3. **Elimination of domain-specific preprocessing**:
Deep learning models do not require domain-specific preprocessing steps like
traditional machine learning models do. This makes them more versatile and
easier to apply across different domains. 4. **Removal of labor-intensive
feature engineering**: Deep learning replaces the need for manual feature
engineering, reducing the amount of human labor required to build a mode

In [17]:
result3 = qa_chain1("What are some challenges or limitations of deep learning algorithms?")

print(fill(result3["result"].strip(),width=80))

Deep learning algorithms have several challenges and limitations that hinder
their ability to fully explain complex phenomena: 1. **Complexity**: Deep neural
networks can be very complex, making it difficult to understand how they make
predictions or decisions. This complexity can lead to overfitting, where the
model becomes too specialized to the training data and fails to generalize well
to new situations. 2. **Interpretability**: Deep learning models often lack
interpretability, meaning it's hard to understand why the model is making a
particular prediction or decision. This makes it difficult to identify and
address errors or biases in the model. 3. **Overfitting**: Deep learning models
can easily overfit the training data, which means they become too good at
fitting the training data and fail to generalize well to new situations.
Overfitting occurs when the model learns the noise in the training data rather
than the underlying patterns. 4. **Lack of understanding**: Despite signif

##Template 2 and second queries

In [18]:
template2 = """
<s>[INST] <<SYS>>
You are a Data Scientist tasked with analyzing the following information to provide insights for the question at the end.
You need to be concise and accurate in your explanation, but try to not convey it to be too advanced, if possible use bullet points
<</SYS>>

{context}

{question} [/INST]
"""

prompt2 = PromptTemplate(template=template2, input_variables=["context", "question"])


qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt2},
)

In [19]:
result4 = qa_chain2("What is deep learning?")

print(fill(result4["result"].strip(),width=80))

Great! Let's dive into the world of deep learning together. Here are some key
points to help you understand what deep learning is: 1. Definition: Deep
learning is a subfield of machine learning that involves training artificial
neural networks to learn and improve on their own by automatically extracting
features from raw data. 2. Key components: Deep learning consists of three main
components - data, algorithms, and hardware. The data refers to the large
amounts of data required to train the models, while the algorithms are the
software programs that enable the models to learn and improve. Finally, the
hardware includes specialized chips and graphics processing units (GPUs)
designed specifically for deep learning computations. 3. Types of deep learning:
There are several types of deep learning techniques, including supervised
learning, unsupervised learning, and reinforcement learning. Supervised learning
involves training models to predict outputs based on inputs, while unsupervised


In [20]:
result5 = qa_chain2("What advantages does deep learning have over traditional machine learning?")

print(fill(result5["result"].strip(),width=80))

Deep learning offers several advantages over traditional machine learning:  1.
**Automatic feature extraction**: Deep learning algorithms learn how to extract
relevant features from raw data without requiring manual engineering. This saves
time and effort compared to traditional machine learning approaches, which often
require extensive feature engineering. 2. **Improved accuracy**: By learning
multiple layers of representations, deep learning models can capture complex
patterns in the data, leading to improved accuracy compared to traditional
machine learning models. 3. **Elimination of domain-specific preprocessing**:
Deep learning models do not require domain-specific preprocessing steps like
traditional machine learning models do. This makes them more versatile and
easier to apply across different domains. 4. **Removal of labor-intensive
feature engineering**: Deep learning replaces the need for manual feature
engineering, reducing the amount of human labor required to build a mode

In [21]:
result6 = qa_chain2("What are some challenges or limitations of deep learning algorithms??")

print(fill(result6["result"].strip(),width=80))

As a Data Scientist, I can explain the challenges and limitations of deep
learning algorithms based on the provided text. Here are some key points:
Challenges in Deep Learning Optimization: 1. Local Minima: Deep learning models
often have complex objective functions that lack closed-form solutions. As a
result, finding the global minimum can be difficult, and local minima may exist.
2. Saddle Points: Another challenge is identifying saddle points, which occur
when the model's parameters converge to a point that is neither a local minimum
nor maximum. 3. Vanishing Gradients: Deep neural networks with many layers can
suffer from vanishing gradients, making it harder to train deeper models.
Insights:  1. Complexity of Objective Functions: Many deep learning objectives
are nonlinear and cannot be solved analytically. Numerical optimization methods
are used instead. 2. Challenges in Finding Global Minimum: Deep learning models
often have complex objective functions that make it hard to find

##Template 3 and third queries

In [22]:
template3 = """
<s>[INST] <<SYS>>
You are an expert in deep learning tasked with analyzing the following information from the context to provide insights for the question.
You need to be concise and accurate in your explanation, but convey it with the intention of teaching data science students.
Format your answer by first giving a short introduction. Then utilize bullet points while going through your explanation.
Lastly sum up the answer with a short conclusion to the question.
<</SYS>>

{context}

{question} [/INST]
"""

prompt3 = PromptTemplate(template=template3, input_variables=["context", "question"])


qa_chain3 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt3},
)

In [23]:
result7 = qa_chain3("What is deep learning?")

print(fill(result7["result"].strip(),width=80))

Introduction: Deep learning is a subfield of machine learning that focuses on
training artificial neural networks to learn complex patterns in data. These
patterns are often too difficult or time-consuming to identify using traditional
machine learning methods. In this answer, we will provide an overview of deep
learning, its key concepts, and its potential benefits. Key Concepts: 1.
Artificial Neural Networks (ANNs): ANNs are computational models inspired by the
structure and function of the human brain. They consist of interconnected nodes
or neurons that process inputs and produce outputs. 2. Multi-Level
Representation Learning: Deep learning represents data using multiple levels of
abstraction, starting from raw inputs and gradually refining them through a
series of transformations. This allows the model to capture increasingly
abstract features and patterns in the data. 3. End-to-End Learning: Deep
learning models aim to learn the mapping between input data and output labels
direc

In [24]:
result8 = qa_chain2("What advantages does deep learning have over traditional machine learning?")

print(fill(result8["result"].strip(),width=80))

Deep learning offers several advantages over traditional machine learning:  1.
**Automatic feature extraction**: Deep learning algorithms learn how to extract
relevant features from raw data without requiring manual engineering. This saves
time and effort compared to traditional machine learning approaches, which often
require extensive feature engineering. 2. **Improved accuracy**: By learning
multiple layers of representations, deep learning models can capture complex
patterns in the data, leading to improved accuracy compared to traditional
machine learning models. 3. **Elimination of domain-specific preprocessing**:
Deep learning models do not require domain-specific preprocessing steps like
traditional machine learning models do. This makes them more versatile and
easier to apply across different domains. 4. **Removal of labor-intensive
feature engineering**: Deep learning replaces the need for manual feature
engineering, reducing the amount of human labor required to build a mode

In [25]:
result9 = qa_chain2("What are some challenges or limitations of deep learning algorithms?")

print(fill(result9["result"].strip(),width=80))

Deep learning algorithms have several challenges and limitations that hinder
their ability to fully explain complex phenomena: 1. **Lack of
interpretability**: Deep learning models are often criticized for being opaque
and difficult to interpret. It can be challenging to understand why a model is
making specific predictions or decisions, which limits our understanding of the
underlying mechanisms. 2. **Overfitting**: Deep learning models can easily
overfit the training data, resulting in poor generalization performance on
unseen data. This occurs when the model becomes too complex and learns the noise
in the training data rather than the underlying patterns. 3. **Computational
cost**: Training deep learning models requires significant computational
resources, including powerful hardware and large amounts of memory. This can
limit the scale of applications where real-time performance is critical. 4.
**Lack of domain knowledge integration**: Deep learning models are typically
trained usi

##Prompt 2 and 3 seems to add a lot more processing time for the execution of the query. This could be due to the gradually building complexity of the prompt.
##The answer also becomes more detailed, especially for the 3rd prompt, which definitely outputs the most detailed answer based on the same queries. This is of course at the cost of the time it takes for the query to process and for this purpose we decided to go with the first prompt for our application, as it will reduce the loading time and the general purpose of the app is just the demonstration.

#Gradio application

In [34]:
import gradio as gr
def rag_qa(query):
    result1 = qa_chain1(query)  # Use query as both context and question
    print("Result from rag_qa function:", result1)  # Print the result
    return result1["result"].strip()

# Create Gradio interface
interface = gr.Interface(fn=rag_qa, inputs="textbox", outputs="text", title="RAG Information Retrieval on Deep Learning", description="Ask about Deep Learning")
interface.queue().launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://098f699d98f5c48a0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
